In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-05-03 14:28:31.469791: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 14:28:32.546777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("../data/flags/flags.csv", header=None)

flags = set([(df[1][i],df[2][i],df[3][i]) for i in range(0,14)])

colors = ["red","lightblue","blue","green","orange","yellow","white"]

x = []
y = []

for c1 in colors:
    x.append(c1)
    if c1 in [x[0] for x in flags]:
        y.append(1)
    else:
        y.append(0)
    
    for c2 in colors:
        x.append(c1+","+c2)
        if c2 in [x[1] for x in flags if x[0]==c1]:
            y.append(1)
        else:
            y.append(0)
        
        for c3 in colors:
            x.append(c1+","+c2+","+c3)
            if c3 in [x[2] for x in flags if x[0]==c1 and x[1]==c2]:
                y.append(1)
            else:
                y.append(0)

#y = np.array(df[0])

#x = np.array([df[1][i]+","+df[2][i]+","+df[3][i] for i in range(0,len(y))])

#x = [df[1][i]+","+df[2][i] for i in range(0,14)]

#y = list(df[3])

x = np.array(x)

y = np.array(y)

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20, split=",")

# updates internal vocabulary based on the lyrics
tokenizer.fit_on_texts(x)

# transform each text in x to a sequence of tokens
x = tokenizer.texts_to_sequences(x)

x = np.array(pad_sequences(x, maxlen = 3))

print(x)

[[0 0 1]
 [0 1 1]
 [1 1 1]
 ...
 [7 7 5]
 [7 7 6]
 [7 7 7]]


In [4]:
num = len(y)

#idx = np.random.permutation(len(x))
#print(idx)
#x = x[idx]
#y = y[idx]

#x_train, x_test = x[:int(num*0.8)], x[int(num*0.8):]
#y_train, y_test = y[:int(num*0.8)], y[int(num*0.8):]

x = x.reshape((x.shape[0], x.shape[1], 1))
#x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
#x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y))

np.unique(y)

array([0, 1])

In [5]:
print(n_classes)

2


In [6]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [7]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)


In [8]:
input_shape = x.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0,
    dropout=0,
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    metrics=["accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x,
    y,
    validation_data=(x,y),
    #validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
)

model.evaluate(x, y, verbose=1)

2023-05-03 14:28:35.810625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 14:28:35.856983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 14:28:35.857713: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 3, 1)        2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 3, 1)        7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

2023-05-03 14:28:46.668084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-03 14:28:47.530648: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1d1af650 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-03 14:28:47.530695: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 960M, Compute Capability 5.0
2023-05-03 14:28:47.537739: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-03 14:28:47.745301: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 21s 104ms/step - loss: 0.3176 - accuracy: 0.9248 - val_loss: 0.2384 - val_accuracy: 0.9248
Epoch 2/100
13/13 [==============================] - 0s 32ms/step - loss: 0.2708 - accuracy: 0.9248 - val_loss: 0.2429 - val_accuracy: 0.9248
Epoch 3/100
13/13 [==============================] - 0s 33ms/step - loss: 0.2510 - accuracy: 0.9248 - val_loss: 0.2322 - val_accuracy: 0.9248
Epoch 4/100
13/13 [==============================] - 0s 33ms/step - loss: 0.2277 - accuracy: 0.9248 - val_loss: 0.2194 - val_accuracy: 0.9248
Epoch 5/100
13/13 [==============================] - 0s 33ms/step - loss: 0.2165 - accuracy: 0.9248 - val_loss: 0.2060 - val_accuracy: 0.9248
Epoch 6/100
13/13 [==============================] - 0s 33ms/step - loss: 0.2172 - accuracy: 0.9248 - val_loss: 0.2032 - val_accuracy: 0.9323
Epoch 7/100
13/13 [==============================] - 0s 33ms/step - loss: 0.2037 - accuracy: 0.9348 - val_loss: 0.2031 - val_accuracy: 0.9248
Epoch 8/100
13/1

[0.15838107466697693, 0.9423558712005615]

In [9]:
model.save('./flags_validator_model')

INFO:tensorflow:Assets written to: ./flags_validator_model/assets


INFO:tensorflow:Assets written to: ./flags_validator_model/assets
